In [20]:
import pickle
import numpy as np
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import tensorflow as tf

In [21]:
with open('X_train_scaled_dic.pickle', 'rb') as f:
    X_train_scaled_dic = pickle.load(f)
with open('X_test_scaled_dic.pickle', 'rb') as f:
    X_test_scaled_dic = pickle.load(f)
with open('y_train_dic.pickle', 'rb') as f:
    y_train_dic= pickle.load(f)
with open('y_test_dic.pickle', 'rb') as f:
    y_test_dic= pickle.load(f)

In [22]:
with open('sector.pickle', 'rb') as f:
    sector = pickle.load(f)

# 윈도우 데이터 셋 생성

In [23]:
# 윈도우 데이터셋 생성 함수
def windowed_dataset(X, y, X_size, y_size, X_shift, y_shift, X_stride, y_stride, batch_size):
    
    # numpy array x(input)를 Dataset으로 변환
    ds_x = tf.data.Dataset.from_tensor_slices(X)
    
    #윈도우셋 구성
    ds_x = ds_x.window(size=X_size,stride=X_stride, shift=X_shift, drop_remainder=True)
    #한 차원 줄야사 반환
    ds_x = ds_x.flat_map(lambda x: x.batch(X_size))
    
    ds_y = tf.data.Dataset.from_tensor_slices(y)
    
    #같은 인덱스에 있는 애들 끼리 묶기
    ds = tf.data.Dataset.zip((ds_x, ds_y))

    
    return ds.batch(batch_size).prefetch(1)

In [24]:
## 파라미터 지정

In [25]:
#윈도우 사이즈
WINDOW_SIZE = 10
# X에 대한 윈도우 함수 설정값
X_size = WINDOW_SIZE
X_shift = 1
X_stride = 1
# y에 대한 윈도우 함수 설정값
y_size = 1
y_shift = 1
y_stride = 1

batch_size = 32

In [26]:
tf_dataset_train_dic={}
for sector_name, sector_df in X_train_scaled_dic.items():
    tf_dataset_train_dic[sector_name] = {}
    for stock_code, df in sector_df.items():
        X_train = df
        y_train = y_train_dic[sector_name][stock_code]
        tf_dataset_train_dic[sector_name][stock_code] = windowed_dataset(X_train,y_train[WINDOW_SIZE:], X_size, y_size,
                                    X_shift, y_shift, X_stride, y_stride, batch_size)

        
        
tf_dataset_test_dic={}
for sector_name, sector_df in X_test_scaled_dic.items():
    tf_dataset_test_dic[sector_name] = {}
    for stock_code, df in sector_df.items():
        X_test = df
        y_test = y_test_dic[sector_name][stock_code]
        tf_dataset_test_dic[sector_name][stock_code] = windowed_dataset(X_test,y_test[WINDOW_SIZE:], X_size, y_size,
                                    X_shift, y_shift, X_stride, y_stride, batch_size)

In [27]:
# 데이터 shape, 미리보기로 체크
for x, y in tf_dataset_train_dic['Food']['097950'].take(1):
    print('X:', x.shape)
    print(x[:3])
    print()
    print('Y:', y.shape)
    print(y[:3])
    print('-'*100)

X: (32, 10, 5)
tf.Tensor(
[[[0.08718198 0.09461967 0.08545455 0.08431373 0.08082707]
  [0.04791903 0.08163265 0.07818182 0.08627451 0.08458647]
  [0.05072111 0.08534323 0.08727273 0.08823529 0.09586466]
  [0.12175059 0.1038961  0.09636364 0.10392157 0.10338346]
  [0.09202992 0.10760668 0.10545455 0.11568627 0.11466165]
  [0.09526414 0.12244898 0.12181818 0.1254902  0.11654135]
  [0.07396998 0.11131725 0.11636364 0.11764706 0.12030075]
  [0.06126101 0.11131725 0.11272727 0.10588235 0.10526316]
  [0.07603778 0.11502783 0.10363636 0.09803922 0.10714286]
  [0.15095811 0.10760668 0.12909091 0.10588235 0.14097744]]

 [[0.04791903 0.08163265 0.07818182 0.08627451 0.08458647]
  [0.05072111 0.08534323 0.08727273 0.08823529 0.09586466]
  [0.12175059 0.1038961  0.09636364 0.10392157 0.10338346]
  [0.09202992 0.10760668 0.10545455 0.11568627 0.11466165]
  [0.09526414 0.12244898 0.12181818 0.1254902  0.11654135]
  [0.07396998 0.11131725 0.11636364 0.11764706 0.12030075]
  [0.06126101 0.11131725 0.1

In [28]:
X_train_scaled_dic['Food']['097950'].iloc[:30]

,거래량,시가,고가,저가,종가
일자,,,,,
2010-06-22,0.087182,0.094620,0.085455,0.084314,0.080827
2010-06-23,0.047919,0.081633,0.078182,0.086275,0.084586
2010-06-24,0.050721,0.085343,0.087273,0.088235,0.095865
2010-06-25,0.121751,0.103896,0.096364,0.103922,0.103383
2010-06-28,0.092030,0.107607,0.105455,0.115686,0.114662
2010-06-29,0.095264,0.122449,0.121818,0.125490,0.116541
2010-06-30,0.073970,0.111317,0.116364,0.117647,0.120301
2010-07-01,0.061261,0.111317,0.112727,0.105882,0.105263
2010-07-02,0.076038,0.115028,0.103636,0.098039,0.107143


In [29]:
y_train_dic['Food']['097950'].iloc[[10,11,12]]

,상향,하향,횡보
일자,,,
2010-07-06,0,1,0
2010-07-07,0,1,0
2010-07-08,1,0,0


In [30]:
for sector_name, sector_tf in tf_dataset_train_dic.items():
    for stock_code, tf_dataset in sector_tf.items():
        for data in tf_dataset.take(1):
            print(stock_code)
            print(f'데이터셋(X) 구성(batch_size, window_size, feature갯수): {data[0].shape}')
            print(f'데이터셋(Y) 구성(batch_size, window_size, feature갯수): {data[1].shape}')

097950
데이터셋(X) 구성(batch_size, window_size, feature갯수): (32, 10, 5)
데이터셋(Y) 구성(batch_size, window_size, feature갯수): (32, 3)
271560
데이터셋(X) 구성(batch_size, window_size, feature갯수): (32, 10, 5)
데이터셋(Y) 구성(batch_size, window_size, feature갯수): (32, 3)
000080
데이터셋(X) 구성(batch_size, window_size, feature갯수): (32, 10, 5)
데이터셋(Y) 구성(batch_size, window_size, feature갯수): (32, 3)
004370
데이터셋(X) 구성(batch_size, window_size, feature갯수): (32, 10, 5)
데이터셋(Y) 구성(batch_size, window_size, feature갯수): (32, 3)
005300
데이터셋(X) 구성(batch_size, window_size, feature갯수): (32, 10, 5)
데이터셋(Y) 구성(batch_size, window_size, feature갯수): (32, 3)
093050
데이터셋(X) 구성(batch_size, window_size, feature갯수): (32, 10, 5)
데이터셋(Y) 구성(batch_size, window_size, feature갯수): (32, 3)
020000
데이터셋(X) 구성(batch_size, window_size, feature갯수): (32, 10, 5)
데이터셋(Y) 구성(batch_size, window_size, feature갯수): (32, 3)
105630
데이터셋(X) 구성(batch_size, window_size, feature갯수): (32, 10, 5)
데이터셋(Y) 구성(batch_size, window_size, feature갯수): (32, 3)
001070
데이터셋(X) 구

036570
데이터셋(X) 구성(batch_size, window_size, feature갯수): (32, 10, 5)
데이터셋(Y) 구성(batch_size, window_size, feature갯수): (32, 3)
005380
데이터셋(X) 구성(batch_size, window_size, feature갯수): (32, 10, 5)
데이터셋(Y) 구성(batch_size, window_size, feature갯수): (32, 3)
000270
데이터셋(X) 구성(batch_size, window_size, feature갯수): (32, 10, 5)
데이터셋(Y) 구성(batch_size, window_size, feature갯수): (32, 3)
012330
데이터셋(X) 구성(batch_size, window_size, feature갯수): (32, 10, 5)
데이터셋(Y) 구성(batch_size, window_size, feature갯수): (32, 3)
033780
데이터셋(X) 구성(batch_size, window_size, feature갯수): (32, 10, 5)
데이터셋(Y) 구성(batch_size, window_size, feature갯수): (32, 3)
009150
데이터셋(X) 구성(batch_size, window_size, feature갯수): (32, 10, 5)
데이터셋(Y) 구성(batch_size, window_size, feature갯수): (32, 3)


# 모델

In [31]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Conv1D, Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tqdm import tqdm


In [32]:
model_dic ={}
for sector_name, stock_list in sector.items():
    model_dic[sector_name] = {}
    for stock_code in stock_list:
        model_dic[sector_name][stock_code] = Sequential([
            # 1차원 feature map 생성
            Conv1D(filters=32, kernel_size=5,
                   padding="causal",
                   activation="relu",
                   #feature의 개수 3개
                   input_shape=[WINDOW_SIZE, 5]),
            # LSTM
            LSTM(16, activation='tanh'),
            Dense(16, activation="relu"),
            Dense(3,activation='softmax'),
        ])

In [33]:
optimizer = Adam(0.0005)
for sector_name, sector_model_dic in model_dic.items():
    for model in sector_model_dic.values():
        model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['acc'])

In [34]:
# earlystopping은 10번 epoch통안 val_loss 개선이 없다면 학습을 멈춥니다.
earlystopping = EarlyStopping(monitor='val_loss', patience=10)

In [35]:
filename_dic={}
for sector_name,stock_list in sector.items():
    filename_dic[sector_name] = {}
    for stock_code in stock_list:
        filename_dic[sector_name][stock_code] = os.path.join('MC',sector_name, stock_code+'ckeckpointer.ckpt')
        
        
with open('filename_dic.pickle','wb') as f:
    pickle.dump(filename_dic,f)

In [36]:
filename_dic['Food']

{'097950': 'MC\\Food\\097950ckeckpointer.ckpt',
 '271560': 'MC\\Food\\271560ckeckpointer.ckpt',
 '000080': 'MC\\Food\\000080ckeckpointer.ckpt',
 '004370': 'MC\\Food\\004370ckeckpointer.ckpt',
 '005300': 'MC\\Food\\005300ckeckpointer.ckpt'}

In [37]:
#체크포인트를 저장    
mc_dic = {}
for sector_name, filename_sector_dic in filename_dic.items():
    mc_dic[sector_name] = {}
    for stock_code, filename in filename_sector_dic.items():
        mc_dic[sector_name][stock_code] = ModelCheckpoint(filename, 
                             save_weights_only=True, 
                             save_best_only=True, 
                             monitor='val_acc',
                             verbose=1)

In [38]:
history_dic={}
# 학습+저장
for sector_name, tf_dataset_dic in tf_dataset_train_dic.items():
    history_dic[sector_name] = {}
    for stock_code, train_data in tf_dataset_dic.items():
        mc = mc_dic[sector_name][stock_code]
        test_data = tf_dataset_test_dic[sector_name][stock_code]
        history = model_dic[sector_name][stock_code].fit(train_data, 
                            validation_data=(test_data), 
                            epochs=80, 
                            callbacks=[mc, earlystopping])
        history_dic[sector_name][stock_code] = history

Epoch 1/80
75/75 [==============================] - 6s 33ms/step - loss: 1.0225 - acc: 0.4383 - val_loss: 0.8663 - val_acc: 0.4796

Epoch 00001: val_acc improved from -inf to 0.47959, saving model to MC\Food\097950ckeckpointer.ckpt
Epoch 2/80
75/75 [==============================] - 1s 17ms/step - loss: 0.8266 - acc: 0.5000 - val_loss: 0.7857 - val_acc: 0.4847

Epoch 00002: val_acc improved from 0.47959 to 0.48469, saving model to MC\Food\097950ckeckpointer.ckpt
Epoch 3/80
75/75 [==============================] - 1s 19ms/step - loss: 0.7906 - acc: 0.4895 - val_loss: 0.7812 - val_acc: 0.5102

Epoch 00003: val_acc improved from 0.48469 to 0.51020, saving model to MC\Food\097950ckeckpointer.ckpt
Epoch 4/80
75/75 [==============================] - 1s 17ms/step - loss: 0.7866 - acc: 0.4744 - val_loss: 0.7791 - val_acc: 0.5170

Epoch 00004: val_acc improved from 0.51020 to 0.51701, saving model to MC\Food\097950ckeckpointer.ckpt
Epoch 5/80
75/75 [==============================] - 1s 17ms/ste

75/75 [==============================] - 1s 15ms/step - loss: 0.7795 - acc: 0.5151 - val_loss: 0.7722 - val_acc: 0.5289

Epoch 00044: val_acc did not improve from 0.53061
Epoch 45/80
75/75 [==============================] - 1s 17ms/step - loss: 0.7800 - acc: 0.5147 - val_loss: 0.7699 - val_acc: 0.5289

Epoch 00045: val_acc did not improve from 0.53061
Epoch 46/80
75/75 [==============================] - 1s 17ms/step - loss: 0.7800 - acc: 0.5139 - val_loss: 0.7686 - val_acc: 0.5323

Epoch 00046: val_acc improved from 0.53061 to 0.53231, saving model to MC\Food\097950ckeckpointer.ckpt
Epoch 47/80
75/75 [==============================] - 1s 19ms/step - loss: 0.7791 - acc: 0.5164 - val_loss: 0.7718 - val_acc: 0.5306

Epoch 00047: val_acc did not improve from 0.53231
Epoch 48/80
75/75 [==============================] - 1s 20ms/step - loss: 0.7793 - acc: 0.5176 - val_loss: 0.7753 - val_acc: 0.5272

Epoch 00048: val_acc did not improve from 0.53231
Epoch 49/80
75/75 [=========================

31/31 [==============================] - 1s 21ms/step - loss: 0.8593 - acc: 0.5128 - val_loss: 0.9260 - val_acc: 0.4454

Epoch 00030: val_acc did not improve from 0.47479
Epoch 31/80
31/31 [==============================] - 1s 18ms/step - loss: 0.8591 - acc: 0.5128 - val_loss: 0.9259 - val_acc: 0.4454

Epoch 00031: val_acc did not improve from 0.47479
Epoch 32/80
31/31 [==============================] - 1s 20ms/step - loss: 0.8589 - acc: 0.5138 - val_loss: 0.9259 - val_acc: 0.4454

Epoch 00032: val_acc did not improve from 0.47479
Epoch 33/80
31/31 [==============================] - 1s 18ms/step - loss: 0.8588 - acc: 0.5138 - val_loss: 0.9259 - val_acc: 0.4454

Epoch 00033: val_acc did not improve from 0.47479
Epoch 34/80
31/31 [==============================] - 1s 19ms/step - loss: 0.8586 - acc: 0.5148 - val_loss: 0.9259 - val_acc: 0.4454

Epoch 00034: val_acc did not improve from 0.47479
Epoch 35/80
31/31 [==============================] - 1s 21ms/step - loss: 0.8585 - acc: 0.5158 - 


Epoch 00017: val_acc did not improve from 0.42687
Epoch 18/80
75/75 [==============================] - 1s 17ms/step - loss: 1.0742 - acc: 0.4123 - val_loss: 1.0504 - val_acc: 0.4269

Epoch 00018: val_acc did not improve from 0.42687
Epoch 19/80
75/75 [==============================] - 1s 17ms/step - loss: 1.0741 - acc: 0.4123 - val_loss: 1.0505 - val_acc: 0.4269

Epoch 00019: val_acc did not improve from 0.42687
Epoch 1/80
75/75 [==============================] - 8s 39ms/step - loss: 0.8451 - acc: 0.4828 - val_loss: 0.7678 - val_acc: 0.5289

Epoch 00001: val_acc improved from -inf to 0.52891, saving model to MC\Food\004370ckeckpointer.ckpt
Epoch 2/80
75/75 [==============================] - 1s 18ms/step - loss: 0.8125 - acc: 0.5042 - val_loss: 0.7620 - val_acc: 0.5289

Epoch 00002: val_acc did not improve from 0.52891
Epoch 3/80
75/75 [==============================] - 2s 22ms/step - loss: 0.8141 - acc: 0.5160 - val_loss: 0.7573 - val_acc: 0.5476s - loss: 0.8120 - acc: 0.51

Epoch 000

75/75 [==============================] - 1s 20ms/step - loss: 1.0685 - acc: 0.3959 - val_loss: 1.1657 - val_acc: 0.2568A: 0s - loss: 1.0682 - acc:

Epoch 00010: val_acc did not improve from 0.33673
Epoch 11/80
75/75 [==============================] - 1s 19ms/step - loss: 1.0676 - acc: 0.4013 - val_loss: 1.1921 - val_acc: 0.2483

Epoch 00011: val_acc did not improve from 0.33673
Epoch 1/80
75/75 [==============================] - 7s 37ms/step - loss: 1.0625 - acc: 0.4043 - val_loss: 1.0299 - val_acc: 0.4405

Epoch 00001: val_acc improved from -inf to 0.44048, saving model to MC\Clothing\020000ckeckpointer.ckpt
Epoch 2/80
75/75 [==============================] - 1s 15ms/step - loss: 1.0515 - acc: 0.4437 - val_loss: 1.0224 - val_acc: 0.4609

Epoch 00002: val_acc improved from 0.44048 to 0.46088, saving model to MC\Clothing\020000ckeckpointer.ckpt
Epoch 3/80
75/75 [==============================] - 1s 18ms/step - loss: 1.0471 - acc: 0.4446 - val_loss: 1.0184 - val_acc: 0.4949

Epoch 00003:

...


Epoch 00010: val_acc did not improve from 0.47619
Epoch 11/80
75/75 [==============================] - 1s 15ms/step - loss: 0.9963 - acc: 0.4517 - val_loss: 0.9148 - val_acc: 0.4762

Epoch 00011: val_acc did not improve from 0.47619
Epoch 12/80
75/75 [==============================] - 1s 14ms/step - loss: 0.9961 - acc: 0.4538 - val_loss: 0.9143 - val_acc: 0.4762

Epoch 00012: val_acc did not improve from 0.47619
Epoch 13/80
75/75 [==============================] - 1s 12ms/step - loss: 0.9959 - acc: 0.4534 - val_loss: 0.9138 - val_acc: 0.4762

Epoch 00013: val_acc did not improve from 0.47619
Epoch 14/80
75/75 [==============================] - 1s 12ms/step - loss: 0.9958 - acc: 0.4492 - val_loss: 0.9136 - val_acc: 0.4762

Epoch 00014: val_acc did not improve from 0.47619
Epoch 15/80
75/75 [==============================] - 1s 14ms/step - loss: 0.9956 - acc: 0.4513 - val_loss: 0.9130 - val_acc: 0.4762

Epoch 00015: val_acc did not improve from 0.47619
Epoch 16/80
75/75 [===============

75/75 [==============================] - 1s 13ms/step - loss: 0.9917 - acc: 0.4530 - val_loss: 0.9076 - val_acc: 0.4830

Epoch 00054: val_acc did not improve from 0.48299
Epoch 55/80
75/75 [==============================] - 1s 13ms/step - loss: 0.9916 - acc: 0.4526 - val_loss: 0.9074 - val_acc: 0.4830

Epoch 00055: val_acc did not improve from 0.48299
Epoch 56/80
75/75 [==============================] - 1s 11ms/step - loss: 0.9915 - acc: 0.4526 - val_loss: 0.9072 - val_acc: 0.4847

Epoch 00056: val_acc improved from 0.48299 to 0.48469, saving model to MC\Manufacturer\000270ckeckpointer.ckpt
Epoch 57/80
75/75 [==============================] - 1s 12ms/step - loss: 0.9914 - acc: 0.4534 - val_loss: 0.9070 - val_acc: 0.4881

Epoch 00057: val_acc improved from 0.48469 to 0.48810, saving model to MC\Manufacturer\000270ckeckpointer.ckpt
Epoch 58/80
75/75 [==============================] - 1s 12ms/step - loss: 0.9913 - acc: 0.4530 - val_loss: 0.9070 - val_acc: 0.4847

Epoch 00058: val_acc did 

75/75 [==============================] - 1s 11ms/step - loss: 0.7861 - acc: 0.5390 - val_loss: 0.7930 - val_acc: 0.5476

Epoch 00014: val_acc did not improve from 0.54762
Epoch 15/80
75/75 [==============================] - 1s 13ms/step - loss: 0.7859 - acc: 0.5374 - val_loss: 0.7927 - val_acc: 0.5493

Epoch 00015: val_acc improved from 0.54762 to 0.54932, saving model to MC\Manufacturer\012330ckeckpointer.ckpt
Epoch 16/80
75/75 [==============================] - 1s 12ms/step - loss: 0.7858 - acc: 0.5378 - val_loss: 0.7925 - val_acc: 0.5493

Epoch 00016: val_acc did not improve from 0.54932
Epoch 17/80
75/75 [==============================] - 1s 13ms/step - loss: 0.7856 - acc: 0.5348 - val_loss: 0.7921 - val_acc: 0.5544

Epoch 00017: val_acc improved from 0.54932 to 0.55442, saving model to MC\Manufacturer\012330ckeckpointer.ckpt
Epoch 18/80
75/75 [==============================] - 1s 12ms/step - loss: 0.7853 - acc: 0.5369 - val_loss: 0.7918 - val_acc: 0.5493

Epoch 00018: val_acc did 

75/75 [==============================] - 1s 10ms/step - loss: 0.9738 - acc: 0.4500 - val_loss: 1.0616 - val_acc: 0.4286

Epoch 00002: val_acc improved from 0.40136 to 0.42857, saving model to MC\Manufacturer\033780ckeckpointer.ckpt
Epoch 3/80
75/75 [==============================] - 1s 12ms/step - loss: 0.9730 - acc: 0.4337 - val_loss: 1.0639 - val_acc: 0.4286

Epoch 00003: val_acc did not improve from 0.42857
Epoch 4/80
75/75 [==============================] - 1s 12ms/step - loss: 0.9715 - acc: 0.4622 - val_loss: 1.0639 - val_acc: 0.4286

Epoch 00004: val_acc did not improve from 0.42857
Epoch 5/80
75/75 [==============================] - 1s 11ms/step - loss: 0.9707 - acc: 0.4584 - val_loss: 1.0627 - val_acc: 0.4286

Epoch 00005: val_acc did not improve from 0.42857
Epoch 6/80
75/75 [==============================] - 1s 11ms/step - loss: 0.9699 - acc: 0.4605 - val_loss: 1.0622 - val_acc: 0.4286

Epoch 00006: val_acc did not improve from 0.42857
Epoch 7/80
75/75 [======================

75/75 [==============================] - 1s 12ms/step - loss: 0.9083 - acc: 0.4639 - val_loss: 0.8538 - val_acc: 0.5255

Epoch 00002: val_acc improved from 0.50340 to 0.52551, saving model to MC\Manufacturer\009150ckeckpointer.ckpt
Epoch 3/80
75/75 [==============================] - 1s 14ms/step - loss: 0.9071 - acc: 0.4421 - val_loss: 0.8577 - val_acc: 0.4609

Epoch 00003: val_acc did not improve from 0.52551
Epoch 4/80
75/75 [==============================] - 1s 13ms/step - loss: 0.9056 - acc: 0.4538 - val_loss: 0.8591 - val_acc: 0.4456

Epoch 00004: val_acc did not improve from 0.52551
Epoch 5/80
75/75 [==============================] - 1s 13ms/step - loss: 0.9047 - acc: 0.4454 - val_loss: 0.8598 - val_acc: 0.4456

Epoch 00005: val_acc did not improve from 0.52551
Epoch 6/80
75/75 [==============================] - 1s 12ms/step - loss: 0.9042 - acc: 0.4505 - val_loss: 0.8602 - val_acc: 0.4456

Epoch 00006: val_acc did not improve from 0.52551
Epoch 7/80
75/75 [======================

In [42]:
with open('label_ratio_dic.pickle', 'rb') as f:
    label_ratio_dic = pickle.load(f)

In [48]:
certificated_stock_dic = {}
stock_acc_dic = {}
for sector_name, sector_model in model_dic.items():
    certificated_stock_dic[sector_name] = []
    stock_acc_dic[sector_name] = {}
    for stock_code, model in sector_model.items():
        s = model.evaluate(tf_dataset_test_dic[sector_name][stock_code])[1]
        stock_acc_dic[sector_name][stock_code] = s
        if s>0.55:
            print(f'{stock_code}: 의 레이블 비율')
            print(np.around(label_ratio_dic[sector_name][stock_code]*100,2))
            print(f'{stock_code} : 의 정확도 : {np.round(s*100,2)}')
            certificated_stock_dic[sector_name].append(stock_code)
        

19/19 [==============================] - 0s 9ms/step - loss: 0.7589 - acc: 0.5782
004370: 의 레이블 비율
상향    49.10
하향    47.89
횡보     3.01
Name: Label, dtype: float64
004370 : 의 정확도 : 57.82
19/19 [==============================] - 0s 8ms/step - loss: 0.6285 - acc: 0.8248
003410: 의 레이블 비율
횡보    87.83
하향     6.89
상향     5.28
Name: Label, dtype: float64
003410 : 의 정확도 : 82.48
19/19 [==============================] - 0s 8ms/step - loss: 0.7867 - acc: 0.5544
010130: 의 레이블 비율
하향    50.43
상향    47.83
횡보     1.74
Name: Label, dtype: float64
010130 : 의 정확도 : 55.44
19/19 [==============================] - 0s 8ms/step - loss: 0.7819 - acc: 0.7126
047040: 의 레이블 비율
횡보    64.18
상향    18.60
하향    17.22
Name: Label, dtype: float64
047040 : 의 정확도 : 71.26
19/19 [==============================] - 0s 8ms/step - loss: 0.9433 - acc: 0.6531
028670: 의 레이블 비율
횡보    54.62
상향    24.88
하향    20.50
Name: Label, dtype: float64
028670 : 의 정확도 : 65.31
12/12 [==============================] - 0s 11ms/step - loss: 0.8733 -

In [49]:
with open('certificated_stock_dic.pickle', 'wb') as f:
    pickle.dump(certificated_stock_dic,f)
with open('stock_acc_dic.pickle', 'wb') as f:
    pickle.dump(stock_acc_dic,f)